Transformer from scratch using pytorch

Following: https://www.kaggle.com/code/arunmohan003/transformer-from-scratch-using-pytorch/notebook

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F 
import math, copy, re
import warnings
import pandas as pd 
import numpy as np 
import seaborn as sns 
import torchtext
import matplotlib.pyplot as plt 
warnings.simplefilter("ignore")
print(torch.__version__)

2.1.2


In [ ]:
class Embedding(nn.Modeule):
    def __init__(self, vocab_size, embedding_dim):
        super(Embedding, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
    
    def forward(self, x):
        """
        x is an inout vector
        returns the embedding vector
        """
        return self.embed(x)

Positional Encoding
Next step is to generate positional encoding. Inorder for the model to make sense of the sentence, it needs to know two things about the each word.

what does the word mean?
what is the position of the word in the sentence.
In "attention is all you need paper" author used the following functions to create positional encoding. On odd time steps a cosine function is used and in even time steps a sine function is used.

Positinal embedding will generate a matrix of similar to embedding matrix. It will create a matrix of dimension sequence length x embedding dimension. For each token(word) in sequence, we will find the embedding vector which is of dimension 1 x 512 and it is added with the correspondng positional vector which is of dimension 1 x 512 to get 1 x 512 dim out for each word/token.

for eg: if we have batch size of 32 and seq length of 10 and let embedding dimension be 512. Then we will have embedding vector of dimension 32 x 10 x 512. Similarly we will have positional encoding vector of dimension 32 x 10 x 512. Then we add both.

In [3]:
# register buffer in Pytorch ->
# If you have parameters in your model, which should be saved and restored in the state_dict,
# but not trained by the optimizer, you should register them as buffers.


class PositionalEmbedding(nn.Module):
    def __init__(self,max_seq_len,embed_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super(PositionalEmbedding, self).__init__()
        self.embed_dim = embed_model_dim

        pe = torch.zeros(max_seq_len,self.embed_dim)
        for pos in range(max_seq_len):
            for i in range(0,self.embed_dim,2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/self.embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/self.embed_dim)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)


    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """
      
        # make embeddings relatively larger
        x = x * math.sqrt(self.embed_dim)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)
        return x

An embedding in a neural network is a relatively low dimensional space into which high dimensional vectors can be translated. Embeddings are used to represent real world objects such as words, images, or videos, in a form that machine learning models can easily process. They reduce the dimensionality of input data while preserving the relationships and structure within the data. 

